# 7. LangSmith を使った RAG アプリケーションの評価


In [1]:
import os


## 7.4. Ragas による合成テストデータの生成


### パッケージのインストール


### 検索対象のドキュメントのロード


In [2]:
from langchain_community.document_loaders import GitLoader


def file_filter(file_path: str) -> bool:
    return file_path.endswith(".mdx")


loader = GitLoader(
    clone_url="https://github.com/langchain-ai/langchain",
    repo_path="./langchain",
    branch="master",
    file_filter=file_filter,
)

documents = loader.load()
print(len(documents))


385


### Ragas による合成テストデータ生成の実装


In [3]:
for document in documents:
    document.metadata["filename"] = document.metadata["source"]


#### 【注意】既知のエラーについて

以下のコードで gpt-4o を使用すると OpenAI API の Usage tier 次第で RateLimitError が発生することが報告されています。

OpenAI API の Usage tier については公式ドキュメントの以下のページを参照してください。

https://platform.openai.com/docs/guides/rate-limits/usage-tiers

このエラーが発生した場合は、以下のどちらかの対応を実施してください。

1. 同じ Tier でも gpt-4o よりレートリミットの高い gpt-4o-mini を使用する
   - この場合、生成される合成テストデータの品質は低くなることが想定されます
2. 課金などにより Tier を上げる
   - Tier 2 で RateLimitError が発生しないことを確認済みです (2024 年 10 月 31 日時点)


In [4]:
import nest_asyncio
from ragas.testset.generator import TestsetGenerator
from ragas.testset.evolutions import simple, reasoning, multi_context
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

nest_asyncio.apply()

generator = TestsetGenerator.from_langchain(
    generator_llm=ChatOpenAI(model="gpt-4o-mini"),
    critic_llm=ChatOpenAI(model="gpt-4o-mini"),
    embeddings=OpenAIEmbeddings(),
)

testset = generator.generate_with_langchain_docs(
    documents,
    test_size=4,
    distributions={simple: 0.5, reasoning: 0.25, multi_context: 0.25},
)


embedding nodes:   0%|          | 0/1210 [00:00<?, ?it/s]

Generating:   0%|          | 0/4 [00:00<?, ?it/s]

In [5]:
testset.to_pandas()


,question,contexts,ground_truth,evolution_type,metadata,episode_done
0,What are the key features and functionalities ...,[# MyScale\n\nThis page covers how to use MySc...,The key features and functionalities of the My...,simple,[{'source': 'docs/docs/integrations/providers/...,True
1,What are the steps involved in the installatio...,[# RWKV-4\n\nThis page covers how to use the `...,The steps involved in the installation and set...,simple,[{'source': 'docs/docs/integrations/providers/...,True
2,What are the two key components for RWKV wrapper?,[# RWKV-4\n\nThis page covers how to use the `...,The two key components for the RWKV wrapper ar...,reasoning,[{'source': 'docs/docs/integrations/providers/...,True
3,How does structured output improve chat model ...,[# Chat models\n\n## Overview\n\nLarge Languag...,Structured output improves chat model interact...,multi_context,[{'source': 'docs/docs/concepts/chat_models.md...,True


### LangSmith の Dataset の作成


In [6]:
from langsmith import Client

dataset_name = "agent-book"

client = Client()

if client.has_dataset(dataset_name=dataset_name):
    client.delete_dataset(dataset_name=dataset_name)

dataset = client.create_dataset(dataset_name=dataset_name)


### 合成テストデータの保存


In [9]:
inputs = []
outputs = []
metadatas = []

for testset_record in testset.test_data:
    inputs.append(
        {
            "question": testset_record.question,
        }
    )
    outputs.append(
        {
            "contexts": testset_record.contexts,
            "ground_truth": testset_record.ground_truth,
        }
    )
    metadatas.append(
        {
            "source": testset_record.metadata[0]["source"],
            "evolution_type": testset_record.evolution_type,
        }
    )

    print("question:", testset_record.question)
    print("contexts:", testset_record.contexts)
    print("ground_truth:", testset_record.ground_truth)
    print("metadata:", testset_record.metadata[0]["source"])
    print("evolution_type:", testset_record.evolution_type)


question: What are the key features and functionalities of the MyScale vector database?
contexts: ['# MyScale\n\nThis page covers how to use MyScale vector database within LangChain.\nIt is broken into two parts: installation and setup, and then references to specific MyScale wrappers.\n\nWith MyScale, you can manage both structured and unstructured (vectorized) data, and perform joint queries and analytics on both types of data using SQL. Plus, MyScale\'s cloud-native OLAP architecture, built on top of ClickHouse, enables lightning-fast data processing even on massive datasets.\n\n## Introduction\n\n[Overview to MyScale and High performance vector search](https://docs.myscale.com/en/overview/)\n\nYou can now register on our SaaS and [start a cluster now!](https://docs.myscale.com/en/quickstart/)\n\nIf you are also interested in how we managed to integrate SQL and vector, please refer to [this document](https://docs.myscale.com/en/vector-reference/) for further syntax reference.\n\nWe 

In [10]:
client.create_examples(
    inputs=inputs,
    outputs=outputs,
    metadata=metadatas,
    dataset_id=dataset.id,
)


## 7.5. LangSmith と Ragas を使ったオフライン評価の実装


### カスタム Evaluator の実装


In [11]:
from typing import Any

from langchain_core.embeddings import Embeddings
from langchain_core.language_models import BaseChatModel
from langsmith.schemas import Example, Run
from ragas.embeddings import LangchainEmbeddingsWrapper
from ragas.llms import LangchainLLMWrapper
from ragas.metrics.base import Metric, MetricWithEmbeddings, MetricWithLLM


class RagasMetricEvaluator:
    def __init__(self, metric: Metric, llm: BaseChatModel, embeddings: Embeddings):
        self.metric = metric

        # LLMとEmbeddingsをMetricに設定
        if isinstance(self.metric, MetricWithLLM):
            self.metric.llm = LangchainLLMWrapper(llm)
        if isinstance(self.metric, MetricWithEmbeddings):
            self.metric.embeddings = LangchainEmbeddingsWrapper(embeddings)

    def evaluate(self, run: Run, example: Example) -> dict[str, Any]:
        context_strs = [doc.page_content for doc in run.outputs["contexts"]]

        # Ragasの評価メトリクスのscoreメソッドでスコアを算出
        score = self.metric.score(
            {
                "question": example.inputs["question"],
                "answer": run.outputs["answer"],
                "contexts": context_strs,
                "ground_truth": example.outputs["ground_truth"],
            },
        )
        return {"key": self.metric.name, "score": score}


In [12]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from ragas.metrics import answer_relevancy, context_precision

metrics = [context_precision, answer_relevancy]

llm = ChatOpenAI(model="gpt-4o", temperature=0)
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

evaluators = [
    RagasMetricEvaluator(metric, llm, embeddings).evaluate
    for metric in metrics
]


### 推論の関数の実装


In [13]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
db = Chroma.from_documents(documents, embeddings)


In [14]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_template('''\
以下の文脈だけを踏まえて質問に回答してください。

文脈: """
{context}
"""

質問: {question}
''')

model = ChatOpenAI(model="gpt-4o-mini", temperature=0)

retriever = db.as_retriever()

chain = RunnableParallel(
    {
        "question": RunnablePassthrough(),
        "context": retriever,
    }
).assign(answer=prompt | model | StrOutputParser())


In [15]:
def predict(inputs: dict[str, Any]) -> dict[str, Any]:
    question = inputs["question"]
    output = chain.invoke(question)
    return {
        "contexts": output["context"],
        "answer": output["answer"],
    }


### オフライン評価の実装・実行


In [16]:
from langsmith.evaluation import evaluate

evaluate(
    predict,
    data="agent-book",
    evaluators=evaluators,
)


View the evaluation results for experiment: 'virtual-limit-31' at:
https://smith.langchain.com/o/0854a5ba-af85-42b8-900e-926caf3e92d9/datasets/14873920-3a2b-4c28-b56a-e63f4740456e/compare?selectedSessions=26669c61-d57d-4885-977b-d786f3388131




0it [00:00, ?it/s]

,inputs.question,outputs.contexts,outputs.answer,error,reference.contexts,reference.ground_truth,feedback.context_precision,feedback.answer_relevancy,execution_time,example_id,id
0,What are the two key components for RWKV wrapper?,[page_content='# RWKV-4\n\nThis page covers ho...,The two key components for the RWKV wrapper ar...,None,[# RWKV-4\n\nThis page covers how to use the `...,The two key components for the RWKV wrapper ar...,1.000000,0.974559,1.535270,09e017c8-eae9-408d-9c3a-ffc1db9e87b4,63b7433c-da19-46d1-b2a7-c9bbc8d4f634
1,What are the steps involved in the installatio...,[page_content='# RWKV-4\n\nThis page covers ho...,The steps involved in the installation and set...,None,[# RWKV-4\n\nThis page covers how to use the `...,The steps involved in the installation and set...,1.000000,0.999999,1.916894,a9dfa4e2-4723-4ea1-8e03-d5aad185d32d,cf9a2ed8-c9ec-4545-b033-4ca687075737
2,What are the key features and functionalities ...,[page_content='# MyScale\n\nThis page covers h...,MyScale vector database offers several key fea...,None,[# MyScale\n\nThis page covers how to use MySc...,The key features and functionalities of the My...,1.000000,0.995958,5.778261,53825c5a-e5e7-421a-bb83-b7a0debb2562,581a8df5-7e37-4d66-ab17-5dabf99b4e1b
3,How does structured output improve chat model ...,[page_content='# Structured outputs\n\n## Over...,Structured output improves chat model interact...,None,[# Chat models\n\n## Overview\n\nLarge Languag...,Structured output improves chat model interact...,0.916667,0.997677,7.126468,aa4b19b3-12c2-469d-bf51-943c27a29e15,b6b18063-28f8-4707-855f-18b1b9d540bc


## LangSmith を使ったオンライン評価の実装


### フィードバックボタンを表示する関数の実装


In [17]:
from uuid import UUID

import ipywidgets as widgets
from IPython.display import display
from langsmith import Client


def display_feedback_buttons(run_id: UUID) -> None:
    # GoodボタンとBadボタンを準備
    good_button = widgets.Button(
        description="Good",
        button_style="success",
        icon="thumbs-up",
    )
    bad_button = widgets.Button(
        description="Bad",
        button_style="danger",
        icon="thumbs-down",
    )

    # クリックされた際に実行される関数を定義
    def on_button_clicked(button: widgets.Button) -> None:
        if button == good_button:
            score = 1
        elif button == bad_button:
            score = 0
        else:
            raise ValueError(f"Unknown button: {button}")

        client = Client()
        client.create_feedback(run_id=run_id, key="thumbs", score=score)
        print("フィードバックを送信しました")

    # ボタンがクリックされた際にon_button_clicked関数を実行
    good_button.on_click(on_button_clicked)
    bad_button.on_click(on_button_clicked)

    # ボタンを表示
    display(good_button, bad_button)


### フィードバックボタンを表示


In [19]:
from langchain_core.tracers.context import collect_runs

# LangSmithのトレースのID(Run ID)を取得するため、collect_runs関数を使用
with collect_runs() as runs_cb:
    output = chain.invoke("LangChainの概要を教えて")
    print(output["answer"])
    run_id = runs_cb.traced_runs[0].id

display_feedback_buttons(run_id)


LangChainは、大規模言語モデル（LLM）を活用したアプリケーションを開発するためのフレームワークです。このフレームワークは、LLMアプリケーションのライフサイクルの各段階を簡素化します。具体的には、以下のような機能があります。

1. **開発**: LangChainのオープンソースコンポーネントやサードパーティの統合を使用してアプリケーションを構築できます。LangGraphを利用することで、状態を持つエージェントを作成し、ストリーミングや人間の介入をサポートします。

2. **生産化**: LangSmithを使用してアプリケーションを検査、監視、評価し、継続的に最適化して自信を持ってデプロイできます。

3. **デプロイ**: LangGraphアプリケーションを生産準備が整ったAPIやアシスタントに変換できます。

LangChainは、LLMや関連技術（埋め込みモデルやベクターストアなど）に対する標準インターフェースを実装しており、数百のプロバイダーと統合されています。また、複数のオープンソースライブラリで構成されており、ユーザーは必要に応じてコンポーネントを選択して使用できます。


Button(button_style='success', description='Good', icon='thumbs-up', style=ButtonStyle())

Button(button_style='danger', description='Bad', icon='thumbs-down', style=ButtonStyle())

フィードバックを送信しました
